<a href="https://colab.research.google.com/github/Jaquark/trek_or_not/blob/master/Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I am a huge fan of Star Trek. I am also a huge fan of giving up a lot of my agency to machines to make my life easier; so why not make a naive bayesian classifier to find out whether or not a television show is Star Trek or not based on plot summaries of the episodes? Because there are better ways to tell if a television show is Star Trek or not - like watching it, or simply reading the title; however the concept of doing it is still a valid way of writing and testing a naive bayesian classifier, which is exactly what we will be doing:

Is it Star Trek (interesting) or Not?


As a bit of an aside, the datasets will be working with will/can be quite large and my computer is quite slow (First generation Pentium i3, an i3-530 at that), so I am going to farm out the processing to Google using this Jupyter notebook and setting up a pyspark cluster.

For the best experience, you can run the entire runtime in a straight go by going to Runtime --> Restar and Run All, or by pressing ctrl+M followed by ctrl+F9

In [1]:
!rm -r spark*
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz     
!tar xf spark-2.4.3-bin-hadoop2.7.tgz
!pip install -q findspark
!curl https://raw.githubusercontent.com/first20hours/google-10000-english/master/google-10000-english-no-swears.txt > most_common_words.txt
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.7"


rm: cannot remove 'spark*': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75153  100 75153    0     0   365k      0 --:--:-- --:--:-- --:--:--  365k


Additionally, we will need access to the wikipedia API; installing, via pip, the module wikipedia will allow us simple parsing;

In [2]:
!pip install -q wikipedia
import wikipedia
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
import string
import math
import shutil
conf=SparkConf().setAppName("interesting").setMaster("local")
sc = SparkContext(conf=conf)

Next we will have some (hardcoded) list of episodes of various TV shows, with a training set of Star Trek episodes (as well as another set of Star Trek episodes to not be in the training set)

I'm going to start with just epsidoes from The Original Series, The Animated Series and The Next Generation; and hope with a limited dataset we can find other star trek episodes from other series.

To me, I am defining that which is interesting as that which is Star Trek. Television shows which have a lot of references to science fiction are more difficult to classify as they have similar sematics. Shows that are homages are incredibly difficult to classify - as in the Futurama episode "Where No Fan Has Gone Before", which is contains all of the original crew of Star Trek except James Doohan, who thought it was stupid, and DeForest Kelley, who was dead.

In [0]:
training_set_star_trek = ["The Man Trap", 	"A Private Little War", 	"Whom Gods Destroy (Star Trek: The Original Series)", #TOS
                          "The Infinite Vulcan", #TAS
                          	"Code of Honor (Star Trek: The Next Generation)", 	"When the Bough Breaks (Star Trek: The Next Generation)", #TNG S1
                            	"A Matter of Honor (Star Trek: The Next Generation)", "The Measure of a Man (Star Trek: The Next Generation)", #S2
                              "Booby Trap (Star Trek: The Next Generation)", "Yesterday's Enterprise (Star Trek: The Next Generation)", #S3
                            "Future Imperfect (Star Trek: The Next Generation)", 	"Clues (Star Trek: The Next Generation)", #S4
                          	"Darmok (Star Trek: The Next Generation)", "The Game (Star Trek: The Next Generation)",
                            "Phantasms  (Star Trek: The Next Generation)", "The Pegasus (Star Trek: The Next Generation)",
                          "Frame of Mind (Star Trek: The Next Generation)", "Ship in a Bottle (Star Trek: The Next Generation)"
                         ]

training_set_is_not_star_trek = ["Hell Is Other Robots", "Fry and the Slurm Factory", "The 30% Iron Chef", "The Honking", "That's Lobstertainment!", "A Head in the Polls",
                                "Time-Flight", "The King's Demons","Childhood's End (Stargate Atlantis)", "33 (Battlestar Galactica)", "Colonial Day", "The Joining (The Outer Limits)"
                                ,"Resurrection (The Outer Limits)", "The Tagger", "Enemies (Stargate SG-1)", "Downloaded (Battlestar Galactica)",
                                "R2 Come Home/Lethal Trackdown"]

set_to_use = { 'trek' : training_set_star_trek, 'not_trek' : training_set_is_not_star_trek }


unknown_but_trek = ["Equilibrium (Star Trek: Deep Space Nine)", "Context Is for Kings"]

unkown_but_not_trek = ["When Aliens Attack", "Children of the Gods", "Full Boyle", "Where No Fan Has Gone Before",
                      ]

prediction_set = { 'trek' : unknown_but_trek, 'not_trek' : unkown_but_not_trek }


first_500 = []
with open('most_common_words.txt') as file:
  first_500 = [next(file).replace('\n','') for x in range(500)]
  


(It took a few runs of the previous and the next block to find out the proper names of the episodes according to wikipedia, copy and paste was not my friend)

Now we want to scrape the plot section of all of those wikipedia pages, additionally we make a corresponding array (for training_purposes) that tells whether or not the plot summary is Star Trek related or not.

The Scrape_Data function has its limitations, as not all television shows posted on Wikipedia share the same page format. Some episodes use the word 'Plot', others use the word 'Synopsis'. All television shows, thus, will be shows that use 'Plot' instead of 'Synopsis'

In [0]:

def scrape_data (set_to_use):

  plot_synopsis = []
  is_trek = []
  episode_name = []
  
  for k in set_to_use.keys():
    for e in set_to_use[k]:
      if k == 'trek':
        is_trek.append(1)
      else :
        is_trek.append(0)
      
      plot = ""
      section = wikipedia.WikipediaPage(title = e).section('Plot')
      section.replace('\n',' ')
      plot += section
      #and then we do some cleanup
      plot = plot.lower()
      plot.replace('(','').translate(str.maketrans("", "", string.punctuation)) 
      
      plot_synopsis.append(plot)
      episode_name.append(e)
      
  return episode_name,plot_synopsis, is_trek


Once we've collected the training data, in the form of a tuple of lists of the same length, we can then create our best fits. Please forgive my over use of global variables. I'm not proud of it.

In the following **fit** function, we take the plot and its trekitude (that is verisimilitude of trek); wherein we calculate the percentages and log values, as well as build a corpus of all words used and the count of words for our two categories - trek or not - necessary to create a naive bayesian classifier.

The work of generating the trek and non trek related words is created by sending the individual plot summaries off to a spark server running on this Colab instance; we simply take all the words and the value one as a tuple, then reduceit on the word value; that is the tuples ('the',1),('the',1);('to',1) will reduce to ('the',2),('to',1); thus giving us our counts of each word for each set.

Additionally, the 500 (or so) most common English words have been stripped from both the individual counts as well as the corpus. Super common words increase the noise.

In [0]:

percentage_of_trek= 0
precentage_of_non_trek = 0
file_name = ["trek.txt","not_trek.txt"]
trek_training_set = {}
not_trek_training_set = {}
corpus = set()
log_known_trek = 0
log_known_non_trek = 0
number_of_trek = 0
number_of_not_trek = 0
corpus = set()

def fit( plot, trek):
  global trek_training_set
  global not_trek_training_set
  global percentage_of_trek
  global precentage_of_non_trek
  global corpus
  global log_known_trek
  global log_known_non_trek
  global number_of_trek
  global number_of_not_trek
  length = len(plot)
  
  #first we need to calculate 
  print(length)
  print(trek)
  for i in trek:
    if i == 1:
      number_of_trek += 1
    else:
      number_of_not_trek += 1
   
  percentage_of_trek = number_of_trek / length
  log_known_trek = math.log(percentage_of_trek)
  precentage_of_non_trek = number_of_not_trek / length
  log_known_non_trek = math.log(precentage_of_non_trek)
  
  #This is where we should do the tokenization that we did earlier above.
  #And use it or loose it, like before, we will be doing this with Spark, unnecessary, but it's a skill like any other
  #continual practice makes one an expert
  
  
  #build the dataset of everything we are using

  for episode,is_trek in zip(plot,trek):
    if is_trek == 1:
      file_idx = 0
    else :
      file_idx = 1
    file = '/content/{0}'.format(file_name[file_idx])
    if os.path.exists(file):
      append_write = 'a' # append if already exists
    else:
      append_write = 'w' # make a new file if not
    
    text_file = open(file, "{0}+".format(append_write))
    text_file.write(episode)
    text_file.write('\n')
    text_file.close()

  for f in file_name:
    file = '/content/{0}'.format(f)
    tf = sc.textFile(file)

    map_training_set = tf.flatMap(lambda line : line.split(" ")).map( lambda x: (x,1))
    map_training_set.saveAsTextFile("map_training_set_{0}".format(f))

    reduce_training_set = map_training_set.reduceByKey(lambda a, b : a + b)
    reduce_training_set.saveAsTextFile("reduce_training_set_{0}".format(f))

    if f == 'trek.txt':
      trek_training_set = reduce_training_set.collectAsMap()
    else:
      not_trek_training_set = reduce_training_set.collectAsMap()
      
    delta_trek = []
    delta_not =[]
    list_of_not_trek =  list(not_trek_training_set.keys())
    list_of_trek = list(trek_training_set.keys())
    
    #this takes forever, how do we fix it?
    for k in trek_training_set.keys():
      corpus.add(k)
  
    for s in not_trek_training_set.keys():
      if s not in list_of_trek:
        delta_trek.append(s)

    for k in not_trek_training_set.keys():
      corpus.add(k)
      
    for s in trek_training_set.keys():
      if s not in list_of_not_trek:
        delta_not.append(s)
         
    for i in delta_not:
      not_trek_training_set[i] = 0
      
    for i in delta_trek:
      trek_training_set[i] = 0
    
    list_of_not_trek =  list(not_trek_training_set.keys())
    list_of_trek = list(trek_training_set.keys())
    
    
    #remove most common words from training sets
    for word in first_500:
      if word in list_of_not_trek:
        del not_trek_training_set[word]
      if word in list_of_trek:
        del trek_training_set[word]
      if word in corpus:
        corpus.remove(word)

With the functionality explained, we can now gather the data:


In [0]:
training_epside,training_plot,training_trek = scrape_data(set_to_use)

Then we can, taking our plots and associated trekitudes, we can do the fitting, thus generating our bayesian classifier model:

In [7]:
#each time we run to ensure data is freshly populated
!rm -r ./map*
!rm -r ./reduce*
!rm -r ./trek*
!rm -r ./not_*

fit(training_plot,training_trek)

rm: cannot remove './map*': No such file or directory
rm: cannot remove './reduce*': No such file or directory
rm: cannot remove './trek*': No such file or directory
rm: cannot remove './not_*': No such file or directory
35
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


With our model now created, we can predict:

We take our set of values we want to test - stored in prediction_set - and we do much the same as we did with the fit function.

However, this time we need to find which value is "stronger" - that is, the sum of all the probabilities words in a given Plot and compare which is bigger. The larger the sum, the more likely that trekitude it is - that means, if we have a value of -600 for Star Trek and a value of -760 for not Star Trek, we can be reasonably sure that it is Star Trek.



In [0]:


def predict(plot_of_unknowns):
  global number_of_trek
  global number_of_not_trek
  trek_or_not = []
  trek_score = []
  not_trek_score = []
  for plot in plot_of_unknowns:
    trek = 0
    not_trek = 0
    word_usage = {}
    
    if os.path.exists('/content/predict_map'):
      shutil.rmtree('/content/predict_map')
    if os.path.exists('/content/predict_reduce'):  
      shutil.rmtree('/content/predict_reduce')
      
    text_file = open('test', "w+")
    text_file.write(plot)
    text_file.write('\n')
    text_file.close()
    
    tf = sc.textFile('test')
    predict_map = tf.flatMap(lambda line : line.split(" ")).map( lambda x: (x,1))
    predict_map.saveAsTextFile("predict_map")

    predict_reduce = predict_map.reduceByKey(lambda a, b : a + b)
    predict_reduce.saveAsTextFile("predict_reduce")
    
    word_usage = predict_reduce.collectAsMap()
    
    for word in word_usage.keys():
      
      if word not in corpus: continue

      log_trained_trek = math.log((trek_training_set[word] + 1) / (number_of_trek + len(corpus)))
      log_trained_not_trek = math.log((not_trek_training_set[word] + 1) / (number_of_not_trek + len(corpus)))
      
      trek += log_trained_trek
      not_trek += log_trained_not_trek
      
    trek += log_known_trek
    not_trek += log_known_non_trek
    

    if trek > not_trek:
      trek_or_not.append(1)
    else:
      trek_or_not.append(0)
          
    trek_score.append(trek)
    not_trek_score.append(not_trek)
    
  return trek_or_not, trek_score,not_trek_score

And once all the above cells have been run, we can see what the outcome of our predictions are

In [13]:
episode_prediction,prediction_plot,training_trek = scrape_data(prediction_set)

predictions,trek,not_trek = predict(prediction_plot)

for episode, training, prediction, ts, nts in zip(episode_prediction, training_trek, predictions,trek,not_trek):
  statement = ''
  if training == prediction:
    if training == 0:
      statement = ' was correctly classified as not Star Trek'
    else:
      statement = ' was correctly classified as Star Trek'
  else:
    statement = ''
    if training == 0:
      statement = ' was incorrectly classified as Star Trek'
    else:
      statement = ' was incorrectly classified as not Star Trek'
      
  print('{0}{1} with a Trek score of {2} and a Not Trek Score of {3}'.format(episode,statement,ts,nts))

Equilibrium (Star Trek: Deep Space Nine) was correctly classified as Star Trek with a Trek score of -649.9599452535407 and a Not Trek Score of -666.857315201658
Context Is for Kings was correctly classified as Star Trek with a Trek score of -526.2954610000764 and a Not Trek Score of -543.1980174571765
When Aliens Attack was correctly classified as not Star Trek with a Trek score of -743.800041265743 and a Not Trek Score of -738.3645966686486
Children of the Gods was correctly classified as not Star Trek with a Trek score of -6.526711744026447 and a Not Trek Score of -6.2353262438129375
Full Boyle was correctly classified as not Star Trek with a Trek score of -494.15062277856083 and a Not Trek Score of -471.97153317104613
Where No Fan Has Gone Before was incorrectly classified as Star Trek with a Trek score of -592.2556046721269 and a Not Trek Score of -600.4838898387595


As we can see - the Deep Space 9 episode and the Discover episode ('Context is For Kings') have both been correctly identified as Star Trek.

The Futurama episode, When Aliens Attack, Stargate SG-1, Children of the Gods, and Brooklyn 99, Full Boyle, epsidoes were correctly identified as not being Star Trek.

And we can see the limitations with the Futurama episode, Where No Fan has gone before, which has a very large overlap with Star Trek itself - as stated earlier.